In [15]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-


import pandas as pd
import numpy as np
import scipy as sp
import patsy
import pystan
#from pystan import StanFit4model

from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
pandas2ri.activate()



import statsmodels.api as sm

import rpy2.robjects as ro
#import pandas.rpy.common as com

pd.set_option('max_columns', 50)






In [2]:

url = "http://stat.columbia.edu/~gelman/arm/examples/arsenic/wells.dat"
wells = pd.read_table(url, sep='\s+', header=0, index_col=0)
wells["dist100"] = wells['dist']/100

y = wells['switch']
wellsNp = wells.as_matrix()
y_mod = wellsNp[:, 0]

y_hat = y.transpose()

y_mean = np.mean(y_mod)

s = sp.special.logit(y_mean)

logit = np.log(y_mean / (1 - y_mean))

f = "~ 0 + dist + arsenic"
mat = patsy.dmatrix(f, wells, return_type='dataframe')
nrow, ncol = mat.shape


data_loo = {'N': nrow , 'P': ncol, 'y': y, 'x': mat, 'a': s }


In [3]:

logistic = '''
data {
  int<lower=0> N; 
  int<lower=0> P;
  int<lower=0,upper=1> y[N];
  matrix[N,P] x; 
  real a;
}
parameters {
  real beta0;
  vector[P] beta;
}
model {
  beta0 ~ student_t(7, a, 0.1);
  beta ~ student_t(7, 0, 1);
  y ~ bernoulli_logit(beta0 + x * beta);
}
generated quantities {
  vector[N] log_lik;
  for (n in 1:N)
    log_lik[n] <- bernoulli_logit_log(y[n], beta0 + x[n] * beta);
}
'''

In [4]:
fit1 = pystan.stan(model_code=logistic, data=data_loo, iter=1000, chains=4)

#print(fit1)

In [5]:
#log_like = pystan.stanfit4model
#print fit1

parameter_name = "log_lik"
type(fit1)
matrix = fit1.extract(pars=parameter_name)
logLik = matrix['log_lik']
#print(matrix['log_lik'])
#np.take(fit1, parameter_name)

In [22]:
import sys
sys.path.insert(0, '/Users/detcitty/Documents/Stan/loo/loo/')

from psisloo import psisloo



In [26]:
test = loo.psisloo(logLik)
print test

(-1969.2572329257655, array([-0.32739442, -0.7185039 , -1.1631651 , ..., -0.8212528 ,
       -0.77313679, -0.60782668]), array([-0.23255912, -0.05524623, -0.06736831, ..., -0.20792367,
       -0.28384624, -0.07717127]))
